In [1]:
from sa import *
import os, json
import random
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

WEIGHT_FILE = "./qm.json"
TEST_INPUT = "./img0.npy"
RANDOM_MEM_GEN = False
DO_PRINT = False
GENERATE_ISA = False
with open(os.path.join(WEIGHT_FILE)) as fp:
    QM_DATA = json.loads(fp.read())
if RANDOM_MEM_GEN:
    TEST_INPUT = np.array([random.randint(-7,7) for i in range(0, -16, -1)])
    np.save("TEST_INPUT.npy", TEST_INPUT)
    TEST_WEIGHT = np.array([
        [[random.randint(-5,5) for j in range(16)] for i in range(16)]
    ])
    np.save("TEST_WEIGHT.npy", TEST_WEIGHT)
else:
    TEST_INPUT = np.load("TEST_INPUT.npy")
    TEST_WEIGHT = np.load("TEST_WEIGHT.npy")


ADDR_MIN = 0x0000_0000
ADDR_MAX = 0x0002_0000
OFF_MEM_WB_BASE_ADDR = 0x0000_0000
OFF_MEM_UB_BASE_ADDR = 0x0001_AA80

FC1_SIZE = 0x0001_8800
FC2_SIZE = 0x0000_2000
FC3_SIZE = 0x0000_0280
UB_SIZE  = 0x0000_0310

FC1_OUTPUT_GOLDEN = np.load("./FC1_GOLDEN_VECTOR.npy")
FC2_OUTPUT_GOLDEN = np.load("./FC2_GOLDEN_VECTOR.npy")
FC3_OUTPUT_GOLDEN = np.load("./FC3_GOLDEN_VECTOR.npy")

Q = 5   # Q-Num format (Q-2.5)

In [2]:
print(TEST_INPUT)
print(TEST_WEIGHT)
RESULT = np.matmul(TEST_INPUT, TEST_WEIGHT.transpose())
print(RESULT.flatten())

[-7 -7  5  4 -5  6  6  2  5 -6  1  7 -7  6  0  3]
[[[ 2 -5 -5  0 -3 -1 -4  4  4 -5  2 -4 -1  4  4 -1]
  [-3  5  0  0 -5  3  3  5  3 -5  3  0  2  5 -5  5]
  [ 2 -1  3 -5  5  0 -4  3  5 -1 -5  0 -1  5 -3  5]
  [ 3 -1 -4  3 -3  2  5  0 -4 -3 -5 -3 -4 -4  4  3]
  [ 5  5 -1  0  3 -3 -2 -4  1 -5  2  2  2  4  0  4]
  [-1  1 -4  3  0  2  3  4  1 -3  3 -4 -1  1  2 -3]
  [ 0  2 -4 -5 -2  0  3 -4  4  0  3  3  1 -1  4  0]
  [ 3 -5 -2  2 -4 -5  2  4 -4  0  2 -4  3  3  5  1]
  [ 5  1 -4  3  1  1  2 -3 -2 -2 -1 -5  0  4 -2  3]
  [-2 -5  4 -3  3  0  1  5 -5  4 -4  3 -2  3  3  5]
  [-5  0  3 -5 -4  4  0  5 -2 -5  5  1 -1 -3 -1 -3]
  [-1  5  5 -1 -3  3  5 -3  4 -4 -5 -2  5  0 -5 -3]
  [ 3 -3  5 -1  4  4 -3  1 -5 -1  2  3 -2 -1  0  0]
  [-4  2  0  2 -2 -1 -5 -3 -3  3  3 -5 -3  5  3  5]
  [-2 -4  3 -2  1  0  5  0  4 -3 -4  4  5 -1 -2  1]
  [ 2 -3  0 -2 -3  3 -2  4 -1  3  1 -5  5  2 -4 -1]]]
[ -10   68  -54   13  -53   16   81  -94   37   27  -54 -130   25  -26
   -4  -39]


In [3]:
# Instantiation
SA = SYSTOLIC_ARRAY(gen_isa=GENERATE_ISA, USE_Q_NUMBER=False, Q=4)
OFF_MEM = BRAM(depth=8192, data_num=16, nbits=8)
OFF_MEM.write(addr=0, val=encode(np.flip(TEST_INPUT), 16, 8))
#TEST_WEIGHT = TEST_WEIGHT.transpose()
TEST_WEIGHT = TEST_WEIGHT[0]
#print(TEST_WEIGHT)
for i in range(16):
    #print(TEST_WEIGHT[i].flatten())
    #print(f"{i}:{TEST_WEIGHT[i]}")
    OFF_MEM.write(addr=i+1, val=encode(TEST_WEIGHT[i].flatten().tolist(), 16, 8))

In [4]:
SA.AXI_TO_UB_INST(OFF_MEM,0,0)
for i in range(16):
    SA.AXI_TO_WB_INST(OFF_MEM, i,i+1)

# Calc
for i in range(20):
    SA.LOAD_WEIGHT(i)
for i in range(4):
    SA.UB_TO_DATA_FIFO_INST(0)

In [5]:
#SA.WB_PRINT()
#SA.DATA_FIFO_PRINT(dec=True)
#SA.MMU_WEIGHT_PRINT(dec=True)
SA.MAT_MUL(0)

[-10, 68, -54, 13, -53, 16, 81, -94, 37, 27, -54, -130, 25, -26, -4, -39]


'f644ca0dcb1051a2251bca8019e6fcd9'

In [6]:
SA.WRITE_RESULT(10, 0)
print(decode(SA.UB.data[10], 16, 8))
print(RESULT.flatten())

[-10, 68, -54, 13, -53, 16, 81, -94, 37, 27, -54, -128, 25, -26, -4, -39]
[ -10   68  -54   13  -53   16   81  -94   37   27  -54 -130   25  -26
   -4  -39]


32x32

In [7]:
if RANDOM_MEM_GEN:
    TEST_INPUT = np.array([random.randint(-7,7) for i in range(0, -32, -1)])
    np.save("TEST_INPUT_1x32.npy", TEST_INPUT)
    TEST_WEIGHT = np.array(
        [[random.randint(-5,5) for j in range(32)] for i in range(64)]
    )
    np.save("TEST_WEIGHT_32x64.npy", TEST_WEIGHT)
else:
    TEST_INPUT = np.load("TEST_INPUT_1x32.npy")
    TEST_WEIGHT = np.load("TEST_WEIGHT_32x64.npy")

print(TEST_INPUT.shape, TEST_WEIGHT.shape)
print(np.matmul(TEST_INPUT, TEST_WEIGHT.transpose()))

(32,) (64, 32)
[-174   45  -78   83 -153   45  -47  -25  -74  -99  -22   30   -5  -28
   75 -148   56  172  -59   32  -23   52  -59   -5   39   34 -155   45
  256   71 -119  -34  123  -27 -102  -60   76  -29   13  -38  -39   53
  106  -41  -21   39   13  -38  120  -40   60   10   33  -36 -108   41
  -74  156   22   44   94   72   69   39]


In [8]:
print(TEST_INPUT)
print(TEST_WEIGHT)

[-7 -3  0  2  4  7  2 -2 -3  3 -3 -2 -3  3 -7 -7 -3  5 -1  6  0  5  0 -4
  3 -5  6 -1 -2  7  5 -7]
[[ 1  2 -4 -2  2  0 -4  5  4  4  3  5  4  0 -4  1  3 -5  2 -3 -2 -2  2 -2
  -5  2  2  0  3 -2 -4  4]
 [-1 -1  1  3 -1  3  2 -4  3 -2  1  4  3 -3 -5 -4 -2  4  3 -2 -5 -4  1 -5
  -5 -3 -3 -1 -5 -5  1 -1]
 [ 1 -2 -2 -2  0 -1  2  4 -1 -4  4 -4  5 -5  0 -5  4  3 -1 -4 -1  3 -1 -3
   2  0 -2 -2  0 -5 -3  1]
 [-4 -1 -1 -1 -1 -1  5  0  5  1 -1  1 -2  4 -5  2 -5  0 -1 -3 -3  1  1  4
   0 -2  3 -4  4  0 -1 -3]
 [ 4  2  2 -5 -1 -1 -3  3  4 -5  5  3 -1  1  5 -3 -5 -3  0 -3 -5 -4  2 -3
   3  4  1  1  5 -4  1 -5]
 [-5 -2  2  5  0  0 -3  0 -2 -1 -5  5 -1 -4  5 -1  1  2  0  1 -1  2  4 -2
  -3  0  3 -2  2  3  1  5]
 [ 1  4  0 -2  4 -3  2 -2 -5  2  3  3  2  4  0  4  0  2 -4  0  1 -3 -3 -5
   5 -2 -4  0  0 -5  5  3]
 [ 2  5 -4  2 -1  0  3  2  5  0 -3  0  5  5 -2  1  3  0  4 -3  4  5  3  2
   2 -5  4  0  5 -1  1  4]
 [-3 -3  5  4  1  2 -3 -4  3  4  2  3  0 -5  4 -3 -4 -1 -3 -2  5 -5  1 -3
   0  4 -3  4  0 -3

In [9]:
# Instantiation
SA = SYSTOLIC_ARRAY(gen_isa=GENERATE_ISA, USE_Q_NUMBER=False, Q=4)
OFF_MEM = BRAM(depth=8192, data_num=16, nbits=8)

# Weigh data
addr = 0
for i in range(0, 32, 16):
    for j in range(0, 64, 16):
        tmp = TEST_WEIGHT[j:j+16]
        #tmp = tmp.transpose()[i:i+16].transpose()
        tmp = tmp.transpose()[i:i+16]
        for k in range(16):
            OFF_MEM.write(addr=addr, val=encode(tmp[k].tolist(), 16, 8))
            addr = addr + 1
        #print(f"[{j}:{j+16}][{i}:{i+16}]")
        #print(tmp)

# Input data
DATA_ADDR_BASE = addr
for i in range(0, 32, 16):
    OFF_MEM.write(addr=addr, val=encode(np.flip(TEST_INPUT[i:i+16]), 16, 8))
    addr = addr + 1



In [10]:
# Write weight to WB
for i in range(128):
    SA.AXI_TO_WB_INST(OFF_MEM, i, i)

# Write data to UB
for i in range(2):
    SA.AXI_TO_UB_INST(OFF_MEM, i, DATA_ADDR_BASE+i)

In [11]:
# Init FIFOs
WEIGHT_ADDR = 0
DATA_ADDR = 0

for i in range(4):
    SA.LOAD_WEIGHT(WEIGHT_ADDR)
    WEIGHT_ADDR = WEIGHT_ADDR + 1

for i in range(4):
    SA.UB_TO_DATA_FIFO_INST(DATA_ADDR)
    DATA_ADDR = DATA_ADDR + 1

In [12]:

for i in range(0, 32, 16):
    ACC_ADDR = 0
    for j in range(0, 64, 16):
        # Load weight
        for k in range(16):
            SA.LOAD_WEIGHT(WEIGHT_ADDR)
            WEIGHT_ADDR = WEIGHT_ADDR + 1
        # Calc
        if (i == 0):
            print("Init")
            SA.MAT_MUL(ACC_ADDR)
        else:
            print("Accum")
            SA.MAT_MUL_ACC(ACC_ADDR)
        ACC_ADDR = ACC_ADDR + 1
    SA.UB_TO_DATA_FIFO_INST(0)

# Write reulst
for i in range(4):
    SA.WRITE_RESULT(DATA_ADDR, i)
    DATA_ADDR = DATA_ADDR + 1

Init
[-37, 64, -24, 56, -123, 16, -36, -26, 27, -25, 39, 11, -41, -101, -34, -60]
Init
[-16, 97, 14, 63, 2, 50, 4, -32, 25, -10, -43, 83, 100, 3, -75, -6]
Init
[52, 40, -73, -100, 54, -32, 17, 4, 19, 3, 29, 3, -9, 29, 16, -87]
Init
[81, 0, 54, -25, 3, 23, -52, -16, -19, 34, 107, 55, 97, 45, 42, -48]
Accum
[-137, -19, -54, 27, -30, 29, -11, 1, -101, -74, -61, 19, 36, 73, 109, -88]
Accum
[72, 75, -73, -31, -25, 2, -63, 27, 14, 44, -112, -38, 156, 68, -44, -28]
Accum
[71, -67, -29, 40, 22, 3, -4, -42, -58, 50, 77, -44, -12, 10, -3, 49]
Accum
[39, -40, 6, 35, 30, -59, -56, 57, -55, 122, -85, -11, -3, 27, 27, 87]


In [13]:
#print(TEST_INPUT)
#print(TEST_WEIGHT.transpose()[16].transpose())
print(np.array(decode(SA.UB.data[4], 16, 8)))
print(np.array(decode(SA.UB.data[5], 16, 8)))
print(np.array(decode(SA.UB.data[6], 16, 8)))
print(np.array(decode(SA.UB.data[7], 16, 8)))

[-128   45  -78   83 -128   45  -47  -25  -74  -99  -22   30   -5  -28
   75 -128]
[  56  127  -59   32  -23   52  -59   -5   39   34 -128   45  127   71
 -119  -34]
[ 123  -27 -102  -60   76  -29   13  -38  -39   53  106  -41  -21   39
   13  -38]
[ 120  -40   60   10   33  -36 -108   41  -74  127   22   44   94   72
   69   39]


In [17]:
print(TEST_INPUT.shape)
print(TEST_WEIGHT.shape)
print(TEST_WEIGHT[0].transpose())
#print(np.matmul(TEST_INPUT, TEST_WEIGHT.transpose()))
print(np.clip(np.matmul(TEST_INPUT, TEST_WEIGHT.transpose()), -128, 128))

(32,)
(64, 32)
[ 1  2 -4 -2  2  0 -4  5  4  4  3  5  4  0 -4  1  3 -5  2 -3 -2 -2  2 -2
 -5  2  2  0  3 -2 -4  4]
[-128   45  -78   83 -128   45  -47  -25  -74  -99  -22   30   -5  -28
   75 -128   56  128  -59   32  -23   52  -59   -5   39   34 -128   45
  128   71 -119  -34  123  -27 -102  -60   76  -29   13  -38  -39   53
  106  -41  -21   39   13  -38  120  -40   60   10   33  -36 -108   41
  -74  128   22   44   94   72   69   39]
